Alvaro Viejo Alonso (100451677) & Rodrigo Oliver Coimbra (100451788)

# Final Practical Work: PageRank over Wikipedia articles

## Practice explanation
This practice will implement the **PageRank** algorithm to build a ranking of Wikipedia web pages based on *quality*.

>The page that gets more outgoing links from other pages will be considered of '*more importance*'.


We can model this behavior after a Markov Chain, where each state represents a webpage, and the edges represent the outgoing links (connections between pages).

### The random surfer model
The solution can be expressed as what is called, **The Random surfer model**. We assume an internet surfer cliking random links and traversing pages. When it had done it long enough, the surfer can record how much time he has spent in each webpage. 

This will approximate the probability landing in each page when randomly surfint the internet. Thus, the pages that are more likely to be visited by a user will coincide to be the most important pages.

As we have modelled the problem as a Markov Chain, and applying the **[Ergodic theorem](https://en.wikipedia.org/wiki/Ergodic_theory)** getting the latter probabilities will exactly coincide with the **stationary** distribution of that Markov Chain.

### Computing PageRank
The PageRank, that is, how much quality is inherent to any webpage, will be computed as the **Stationary distribution probabilities**, which represents how much time a user is likely to spend on a certain webpage.

We will compute these stationary distribution in *brute-force* manner, by simulating the document traversing through the multiplication of the **transition matrix** with the **webpage probability vector**. When the difference between the probability vector at time $n-1$ and $n$ is less than a certain threshold $\epsilon$, we can assume that we have reached the stationary distribution.
 $$r_n = M \cdot r_{n-1} \equiv |r_{n-1}-r_{n}| \leq \epsilon \implies r_n \mathtt{\ stationary}$$

### Damping factor
As explained in the latter paragraphs, we are using the *Random surfer model*. The problem is that if this random surfer reaches a page that has no outgoing links, which can happen, its search will stagnate. This is what is defined as a **Rank Sink**.

To help avoid Rank sinks, we will set a probability of randomly jumping to any document in the database. This probability will be known as the damping factor $\beta$.

When visiting a document there will be a probability $\beta$ of visiting a link inside the document, and a probability $(1-\beta)$ of visiting a random document.

This additionally solves for the requirement for the **Ergodic** theorem, that a Markov Chain must be fully connected to have a Stationary distribution.

### Computing PageRank
In order to compute PageRank, we will have to build the transition Matrix $M$, containig the links which are connected.

>NOTE: This matrix $M$ will not represent the fully connected transition matrix, as storing this in memory is highly inneficient. Instead, only the original links are stored in a Sparse Matrix.

An initial uniform vector $R_0$ with probability values of $\frac{1}{N}$ will be assumed.

>N stands for the number of webpages.

This vector will be updated in every iteration of the algorightm by being multiplied by the transition matrix.

$$r_{n} = \beta \cdot ( M \cdot r_{n-1}) + (1-\beta) \cdot u_{Nx1}$$

>$(1-\beta) \cdot u_{Nx1}$ is the responsible for applying the damping factor, as our matrix $M$ is sparse.

#### Injecting lost probability
$r_{n}$ represents a vector of probabilities, thus, it's L1 norm (manhattan distance) must always be one.

There is a posibility, especially for low values of N, that due to the sparsity of matrix M, the L1 norm of the updated vector $r$ is less than 1.

In this case we have to '*inject*' the lost probability. We will compute $d$, which corresponds with probability that is missing, and we will distribute it evenly through all the entries.

$$d = ||r_n||_1 - ||r_{n-1}||_1$$
$$r_{n} = r_{n} + d \cdot u_{Nx1}$$

### Error tolerance
In order to compute the stationary distribution, we would technically have to update the vector $r$ infinitely. 
As we cannot operate for eternity, we will compute the what is the difference between $r_n$ and $r_{n-1}$.
When this difference, which we will call $\gamma$, is small enough, we will assume that the stationary distribution has been reached.

### Convergence
Depending on the size of $N$, it will take more or take more or less iteratios to achieve convergence.

In case of no convergence being reached, the loop will exit in the specified 20 iterations.

## Implementation
The first thing we will have to do is to load in some libraries.
Apart from the typical python packages as `numpy`, `typing` and `pandas`, we will also have to import `re` to extract the outgoing links from each of the pages.

### Data structures
In order to make the implementation as efficient as possible we will use Spark's distributed data structures.

These are:
- `CoordinateMatrix`
- `DenseMatrix`
- `IndexedRowMatrix`

Both `CoordinateMatrix` and `IndexedRowMatrix` are **sparse** datastructures. That is, they are expected to hold matrices with potentially lots of zeros. In order to be as efficient as possible, only the non-zero values of the matrices are stored, which is perfect for our use case as we are only interested in non-zero values (connections between documents).

In the other hand, `DenseMatrix` will be used to hold the PageRank value vector, as there will be NO document containing a PageRank of zero. Additionally, `IndexedRowMatrix` has a built-in method to be multiplied by `DenseMatrix`.

In [ ]:
# The PySpark-specific ones
from pyspark.sql.types import *
from pyspark.sql.functions import *

# Used for creating indeces
from pyspark.sql.window import Window

# PySpark types: Both for building the data structures and typing functionallity
from pyspark.mllib.linalg.distributed import (CoordinateMatrix, MatrixEntry,
                                              DenseMatrix, IndexedRowMatrix, IndexedRow)

# The general Python ones
import pandas as pd
import re
import numpy as np
from typing import Union, List


To test our algorithms, we will be using a sample of the entire dataset. This sampling will be completely random, so in order to ease the data transformations, we will create a new index for the samples that will correspond to their index in the matrix $M_ij$.

This is done due to the fact that wikipedia indices will go very out of bounds for sampled dataset. For example, let's imagine the case that 576 observations are sampled. We will indexes from 0 to 575 in within the samples in order to build the matrix $M$. These new indexes will of course be assigned after the sampling.

To make handling of data easier for the user when testing, all parameters can be modified quickly with a parameter being the `sample_frac` which stores the proportion of the original data that is to be used by the subroutine, a `sample_seed` to allow for reproducibility and then the sample dataset will be programmatically generated.

In [ ]:
# declare the size of the fraction of the data
sample_frac = 0.5#%
sample_seed = 0

# dataset source
data_src = "dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet"

# Add a new index over the window of the dataframe
w = Window.orderBy("id")

# Create our sampled Dataset
wikipediaDF = (spark.read.parquet(data_src) # load the data
              .sample(fraction=sample_frac/100, seed=sample_seed) # take a sample of the dataset
              .withColumn("db_id", row_number().over(w) - 1) # create the new index
              .cache()) # cache this dataset, as all data transformations will be done throught
                        # this dataset

## Inspection of the data

It is of paramount importance to analyze and understand the data we are dealing with. We can see that the dataset has the following structure: 

- `title` 
- `id` 
- `revisionId` 
- `revisionTimestamp` 
- `revisionUsername` 
- `revisionUsernameId`
- `text`
- `db_id`

For this practice we are only interested in `title` (string), `id` (integer)[original index], `db_id` (integer)[sample index] and `text` (string). It is possible to see that the `text` column contains the entire wiki-code of the page, so we will be parsing it to extract the relevant internal wikipedia links.

In [ ]:
# Display five elements from wikipediaDF and visualize its structure
display(wikipediaDF.limit(5))

title id revisionId revisionTimestamp revisionUsername revisionUsernameId text db_id Asterales 789 651654361 2015-03-16T16:50:22.000+0000 DMacks 712163 {{Use dmy dates|date=June 2013}} {{More footnotes|date=February 2012}} {{automatic taxobox | image = A_sunflower.jpg | image_caption = [[Sunflower]], ''Helianthus annuus'' | ordo = '''Asterales''' | authority = [[Johann Heinrich Friedrich Link|Link]] {{Cite journal |last=Angiosperm Phylogeny Group |year=2009 |title=An update of the Angiosperm Phylogeny Group classification for the orders and families of flowering plants: APG III |journal=Botanical Journal of the Linnean Society |volume=161 |issue=2 |pages=105–121 |url=http://www3.interscience.wiley.com/journal/122630309/abstract | format= PDF |accessdate=2013-07-06 |doi=10.1111/j.1095-8339.2009.00996.x }} | subdivision_ranks = [[Family (biology)|Families]] | subdivision = *[[Alseuosmiaceae]] *[[Argophyllaceae]] *[[Asteraceae]] - Daisies *[[Calyceraceae]] *[[Campanulaceae]] (incl. [[Lobeliaceae]]) - Bellflowers *[[Goodeniaceae]] (incl. [[Brunoniaceae]]) *[[Menyanthaceae]] *[[Pentaphragmataceae]] *[[Phellinaceae]] *[[Rousseaceae]] (incl. [[Carpodetaceae]]) *[[Stylidiaceae]] (incl. [[Donatiaceae]]) }} '''Asterales''' {{IPAc-en|æ|s|t|ə|ˈ|r|eɪ|l|iː|z}} ''Botanical Latin'', William T Stearn, Timber Press 2004, ISBN 0-88192-627-2 is an [[Order (biology)|order]] of [[dicotyledon]]ous [[flowering plant]]s that includes the large [[Family (biology)|family]] [[Asteraceae]] (or Compositae) known for composite flowers made of [[Floret#floret|florets]], and ten families related to the Asteraceae. The Families and Genera of Vascular Plants, Edited by K. Kubitzki, Volume VIII, Flowering Plants Eudicots Asterales; Springer 2006 The order is a [[Cosmopolitan distribution|cosmopolite]] (plants found throughout most of the world including desert and frigid zones), and includes mostly [[Herbaceous plant|herbaceous]] species, although a small number of trees (such as the [[Lobelia deckenii|giant Lobelia]] and the [[Dendrosenecio|giant Senecio]]) and [[shrub]]s are also present. Asterales are organisms that seem to have evolved from one common [[ancestor]]. Asterales share characteristics on [[Morphology (biology)|morphological]] and biochemical levels. [[Synapomorphies]] (a character that is shared by two or more groups through evolutionary development) include the presence in the plants of [[oligosaccharide]] [[inulin]], a nutrient storage molecule used instead of [[starch]]; and unique [[stamen]] morphology. The stamens are usually found around the [[Gynoecium|style]], either aggregated densely or fused into a tube, probably an [[adaptation]] in association with the plunger (brush; or secondary) [[pollination]] that is common among the families of the order, wherein pollen is collected and stored on the length of the pistil. == Families == The order '''Asterales''' currently includes 11 families, the largest of which are the [[Asteraceae]], with about 25,000 species, and the [[Campanulaceae]] ("bellflowers"), with about 2,000 species. The remaining families count together for less than 500 species. The two large families are cosmopolitan, with many of their species found in the Northern Hemisphere, and the smaller families are usually confined to Australia and the adjacent areas, or sometimes South America. Only the Asteraceae have composite flower heads; the other families do not, but share other characteristics such as storage of inulin that define the 11 families as more closely related to each other than to other plant families or orders such as the [[rosids]]. The name and order Asterales is botanically venerable, dating back to at least 1926 in the [[Hutchinson system| Hutchinson system of plant taxonomy]] when it contained only five families, of which only two are retained in the APG III classification. Under the [[Cronquist system]] of taxonomic classification of flowering plants, [[Asteraceae]] was the only family in the group, but newer 

We will need the size of the sampled data to build the matrix $M$.

In [ ]:
# Count the number of elements in the dataframe
N = wikipediaDF.count()
print(N)

29263


## Creating the transition matrix $M$


### Extracting the links
In this section we will parse the wiki-code of the `text` fields in search of outgoing links.
We will extract the links using the following regular expression:
```python
expr = r"\[\[(.*?)(?:#.*?)?(?:\|.*?)?\]\]"
```
Which will only capture the links. It will remove both **link renames** (text followed after `|` character) and **file headings** (text followed after `#` character).

In order to avoid conflicts in article namings, all titles and links will be transformed into lower case.

In [ ]:
def parse_links(document_body,
                regexp: str = r"\[\[(.*?)(?:\#.*?)?(?:\|.*?)?\]\]"
                ) -> List[Union[str, None]]:
    """Function that given some texts, filters it and extracts valid internal 
    Wikipedia links."""

    data = re.findall(regexp, document_body)
    return list(map(lambda s: s.lower(), data))


Now that we have defined our link parser function, we can transform it into a `udf` in order to use it inside `pyspark.sql`:

In [ ]:
# This function is registered as a UDF - user-defined function in order to be used inside of
# a `select()` statement
parse_links_udf = udf(parse_links, ArrayType(StringType(), False))

### Creating document-link pairs
We will be most interested in obtaining the ID of the pages that are referenced by a particular page, which is itself also identified by its ID. In order to provide a scalable solution we ditched the original concept that employed a UDF and a Pandas Dataframe, due to its lack of distributed computation capability, and instead chose to perform an *intelligent* join using the distributed tools provided by PySpark.

The goal is to build a dataframe that contains id coordinates $(i,j)$ where $i$ is a certain document and $j$ is a document referenced by $i$.

An example would be:
|db_id|	outgoing_id|
|:-:|:-:|
|28940|	0|
|28908|	0|
|28780|	0|

Where we can see that the page with **sampled id** 0 is being referenced by 3 other pages.

<!-- The first dataframe, called `id_link_join_lookupDF` is responsible for storing the relationship between one page (identified by its ID) and the pages to which it links to (also identified by their ID). In order to achieve this we extract the `db_id` from the `wikipediaDF` dataframe and parse the `text` field which is henceforth referred to as `links` after being correctly parsed; the `explode()` method is used to create id-link pairs for each ORIGIN PAGE-REFERENCED PAGE relationship.

The second dataframe, called `id_title_lookupDF` is responsible for storing the one-to-one relationship between the page title (`title`) and IDs (`outgoing_id`). In practice this will be the *dictionary* that we will be using for our *intelligent* join.

Now that we have both dataframes we perform the left join operation with the condition that `id_title_lookupDF.title == id_link_join_lookupDF.col` that takes advantage of the one-to-one correspondence between page title and page IDs. -->

In [ ]:
# NOTE: Create and cache a dataframe containing articles identified by their new ids and
# the list of pages they point to, also identified by their ids
# Note that thanks to using explode, each row will contain and id and a link
id_link_join_lookupDF = (wikipediaDF.select(
    "db_id", # first column will be the sample id of each page
    explode(parse_links_udf("text").alias("links"))) # second column will be the link in 
    .cache())                                        # text form the page contains

# Create the 'dictionary' dataframe that relates page titles to their ids.
# NOTE: This is a very useful dataframe, as it will allow us to join it to the above dataframe
# and  map link titles to id's without using an udf
id_title_lookup_DF = (wikipediaDF.select("db_id", lower("title").alias("title"))
                      .withColumnRenamed("db_id", "outgoing_id"))


# NOTE: Perform the join operating using the aforementioned condition
# and cleaning the result from NULL values (which mean that the links found are 
# not a wikipedia page link), we then remove duplicates and we make sure that 
# no documents are pointing to themselves, as those relations do not count.
# This dataframe will be cached as it will be used to build the matrix M.
docLinkPairs = (id_title_lookup_DF.join( # Perform a join of the two latter dataframes.
                        id_link_join_lookupDF,
                        id_title_lookup_DF.title == id_link_join_lookupDF.col, #join on title match
                        "left")
                .na.drop().select("db_id", "outgoing_id") # drop invalid links
                .distinct() # delete duplicated entries
                .where("db_id != outgoing_id") # remove documents pointing to themselves.
                .cache())

display(docLinkPairs)

db_id,outgoing_id
28940,0
28908,0
28780,0
28754,0
28420,0
28382,0
26917,0
26911,0
24858,0
18152,0


### Creation of the transition matrix $M$

The transition matrix will be a sparse matrix as this is the most convenient and efficient way to store the data given its nature.
As mentioned above, the `CoordinateMatrix` and `IndexedRowMatrix` datatypes will be used. This is because our data is right now in coordinate form, so we will create a `CoordinateMatrix` with it, to later transform it into an `IndexedRowMatrix` to be able to multiply with it.

#### Create the link count dataframe
As we have to distribute the transition equally between links within the same page, we will have to know how many does each page have.

For example, if page `0` links to pages `1`, `2` and `3`, each of its transition probabilities will be $\frac{1}{N_0} = \frac{1}{3}$

So the link count dataframe will be a dataframe containing $(i, N_i)$ pairs.

In [ ]:
# We create the link_countDF which contains the number of references per page
link_countDF = (docLinkPairs.groupBy("db_id") # group all the id ($i$) occurances
                .count() # count how many links belong to each id ($N_i$)
                .withColumnRenamed("count", "link_count"))
# Display the dataframe
display(link_countDF)

db_id,link_count
28940,1
28908,1
28780,1
28754,1
28420,1
28382,1
26917,1
26911,1
24858,1
18152,1


### Explanation of the implementation notation
We could say that the equation for updating the PageRank vector may be divided into two big parts:

$$r_{n} = \underbrace{\beta \cdot ( M \cdot r_{n-1})}_{\text{Left}} + \overbrace{(1-\beta) \cdot u_{Nx1}}^{\text{Right}}$$

And this is the notation used in the `updateR()` function used for each iteration.

In [ ]:
def getTransitionMatrix(docLinkPairs=docLinkPairs, 
                        link_countDF=link_countDF, N=N, 
                        transpose=True) -> CoordinateMatrix:
    """
    Function that create the transition matrix $M$ using the docLinkPairs dataframe
    """
    
    # Join the table with the information of how many outgoing links ($N_i$)
    # each document has. This will produce an Rdd containing coordinate tuples
    # with the form (i, j, probability of transition from i to j)
    docLinkPairsCounts = (docLinkPairs.join(link_countDF, "db_id", "leftouter"))
    coordinates = (docLinkPairsCounts.select(
                            "db_id", "outgoing_id", 
                           (1/docLinkPairsCounts.link_count).alias("probability"))
                   .rdd)
    
    # Convert the coordinate tuples into a Coordinate matrix
    matrix = CoordinateMatrix(coordinates.map(lambda coords: MatrixEntry(*coords)), N, N)
    
    # Transpose the matrix if necessary, (which in our case it is), and transform it into
    # an IndexedRowMatrix to be able to multiply it.
    if transpose:
        return matrix.transpose().toIndexedRowMatrix()
    else:
        return matrix.toIndexedRowMatrix()
    
          
def getRZero(N = N) -> DenseMatrix:
    """This function creates the R PageRank vector and initializes it
    to assign the uniform value of 1/N for each of its elements"""
    # NOTE: It must be a DenseMatrix in order to be multiplied with
    # an IndexedRowMatrix
    return DenseMatrix(N, 1, np.repeat(1/N,N))


def L1norm(R: Union[DenseMatrix, np.ndarray]) -> float:
    """
    This function computes the L1 norm and rounds to result
    to two decimal places
    """
    # NOTE: 2 decimal places where used, as that is what Google Founders
    # used in their paper

    if not isinstance(R, np.ndarray):
        R = R.toArray()

    return np.round(np.linalg.norm(R, ord=1),2)


def multiplyRvecByScalar(R: IndexedRowMatrix, scalar: float) -> IndexedRowMatrix:
    """
    Function used for the purpose of multiplying β * (M @ R) 
    [applying the damping coefficient in a distributed manner]
    """
    return R.rows.map(lambda row: IndexedRow(row.index, row.vector * scalar))


def updateR(R: DenseMatrix, M: IndexedRowMatrix, N=N, beta=0.85) -> DenseMatrix:
    # compute the `left` part of the equation
    left = multiplyRvecByScalar(M.multiply(R), beta)

    # compute the `right` part of the equation
    right = np.repeat((1-beta)/N, N)
    
    # NOTE: This for loop is a very short loop than injects the values from the LEFT
    # vector into the RIGHT vector, essentially summing them to into the new r vector
    for row in left.collect(): right[row.index] = right[row.index] + row.vector[0]

    # Compute how many probability has been lost.
    # NOTE: should be d = L1norm(R) - L1norm(right), but L1norm(R) is assured to be 1.0
    d = 1.0 - L1norm(right)
    
    # inject the lost probability back in an uniform way
    result_vals = right + (d / N)
    
    # return the new r vector into a dense matrix form.
    return DenseMatrix(N, 1, result_vals)

## Executing the loop
Once all the functions have been defined, we just have to join them in a main algorithm loop. This loop will compute the updates of the $r$ vector until the stationary distribution has been reached, or the maximum number of iterations has been surpassed.

In [ ]:
# Create the initial R vector and matrix M
M = getTransitionMatrix()
R0 = getRZero()

# Set the execution parameters
MAXITERS = 20
epsilon = 10e-8 # Set the tolerance of convergance

# set the starting number of iterations to zero
iterN = 0

# set a big enough starting gamma to assure
# the first iteration is always done
gamma = 99

# set the starting R to be R0
R = R0

As it can be seen, both the matrix $M$ and PageRank array $r$ have been created in the previous cell. We can inspect that $M$ is indeed a Sparse Matrix, and that $r$ is a uniform vector.

In [ ]:
print(M.rows.take(5))
print(R.toArray())

[[3.41728463e-05]
 [3.41728463e-05]
 [3.41728463e-05]
 ...
 [3.41728463e-05]
 [3.41728463e-05]
 [3.41728463e-05]]


### Main algorithm loop 🔁
This is where ALL the pageRank computations will take place.

In [ ]:
# execute the algorithm while convergence has yet not been reached,
# and we are still bellow of the maximum number of iterations
while gamma > epsilon and (iterN < MAXITERS):
    # store the old vector for computing the gamma
    Rold = R
    # run the R update
    R = updateR(R, M)
    # increase the number of iterations
    iterN += 1
    # compute the new gamma to check if convergence has been reached
    gamma = L1norm(R.toArray() - Rold.toArray()) 

In [ ]:
# The loop goes on for a maximum of 20 iterations
if iterN >= 20:
    # alert if the maximum number of iterations has been reached
    print("[WARNING] Max number of iterations reached!")

# Inform on how many iterations have been performed
print(f"[INFO]: Finished in {iterN} iterations")

[INFO]: Finished in 10 iterations


## Presenting the data in a pandas dataframe as requested
As requested in the problem statement, we will present the data int a pandas dataframe containing 
- The wikipedia page `id`
- The wikipedia page `title`
- Its associated PageRank value

Additionally, we will scale the PageRank values as the original paper of 1998 suggests, rounding the result to 2 decimals

In [ ]:
def applyPageRankScaling(R: DenseMatrix, rankCoeff=1_000_000) -> np.ndarray:
    """This function makes the output resemble that of the original paper in 1998
    by Page, Lawrence and Brin, Sergey and Motwani, Rajeev and Winograd, Terry"""
    return np.round(R.toArray() * rankCoeff, 2)

# The results are saved in a Pandas Dataframe that includes
# the original id, title and PageRank as request in the project statement
results_pdf = wikipediaDF.select("id", "title").toPandas()
results_pdf["PageRank"] = applyPageRankScaling(R)

In [ ]:
# Display the results using a Pandas Dataframe (as specified in the project statement)
display(results_pdf)

id,title,PageRank
789,Asterales,562.27
1027,August 9,28.02
1359,Anbar (town),28.02
3354,Berlin,5655.86
4051,Brian Kernighan,301.53
5554,Demographics of Costa Rica,28.02
6431,Charles Farrar Browne,28.02
7519,Convolution,292.04
7952,Digital Equipment Corporation,301.53
8233,Death metal,230.48
